In [1]:
#!pip install airbyte

In [2]:
import airbyte as ab

import json
import pandas as pd
import numpy as np

from vectara import index_document

In [3]:
src = ab.get_source('source-airtable', config={'api_key': '<YOUR_AIRTABLE_API_KEY>'})
streams = src.get_available_streams()
streams

['hotel-reviews/imported_table/tblGpuhPqCt1T7vKE']

In [4]:
df = pd.DataFrame([x.metadata for x in  src.get_documents(streams[0])])
display(df.head(5))

## Read Progress

Started reading at 03:47:33.

Read **35,690** records over **1min 13s** (488.9 records / second).




------------------------------------------------


Completed `source-airtable` read operation at 20:48:47.

,_airtable_id,_airtable_created_time,_airtable_table_name,address,categories,city,country,latitude,longitude,name,...,reviews.date,reviews.dateadded,reviews.dorecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.usercity,reviews.username,reviews.userprovince
0,rec0017cxZgRkIz8X,2024-03-16T16:15:38.000Z,Imported table,835 S Chestatee St,"Wedding Sites,Hotels,Hotels & Motels,Meeting F...",Dahlonega,US,34.524950,-83.975840,Holiday Inn Express Dahlonega,...,2016-05-29T00:00:00.000Z,2016-11-05T01:17:49.000Z,None,None,4.0,I travel a lot. Both for business and pleasure...,Personable and professional staff with a wonde...,Roswell,Steve W,GA
1,rec002DwLYmFV1hNR,2024-03-16T16:48:49.000Z,Imported table,3940 Centerview Dr,"Hotels,Family-Friendly Hotels,Budget Hotels,Ho...",Chantilly,US,38.897650,-77.431590,Wingate By Wyndham Chantilly / Dulles Airport,...,2015-08-10T00:00:00.000Z,2016-11-02T21:57:45.000Z,None,None,4.0,Nice enough place to stay in town. It's near p...,"Older hotel, but in a nice enough area.",None,A Traveler,None
2,rec00EILtFOGqiONM,2024-03-16T16:14:14.000Z,Imported table,442 Walker Bay Blvd,Hotels,Walker,US,47.091240,-94.571720,Country Inn Walker,...,2014-06-19T00:00:00.000Z,2016-11-06T18:29:10.000Z,None,None,5.0,I really liked everything that the hotel had t...,Fantastic service,Waukegan,Hello!! :D,IL
3,rec00F5kePHnsajoi,2024-03-16T16:49:16.000Z,Imported table,3243 S Ridge Dr S W,"Hotels,Hotel",Cedar Rapids,US,41.946156,-91.672390,Red Roof Inn Cedar Rapids,...,2015-09-18T00:00:00.000Z,2017-04-12T07:43:15.000Z,None,None,4.0,"Easy to get to,right off the interstate. Check...",Budget style hotel..,Ladson,Richard,SC
4,rec00H8lMws4k3Z71,2024-03-16T16:15:15.000Z,Imported table,105 Melcon Ln,"Hotels & Motels,Lodging,Hotels And Motels",London,US,37.145106,-84.108375,Econo Lodge,...,2015-07-31T00:00:00.000Z,2016-12-13T12:13:12.000Z,None,None,3.0,"Upon arrival, young lady who checked me in was...",Not terrible,Crossville,TJFrench51,TN


In [5]:
print(list(df.columns))

['_airtable_id', '_airtable_created_time', '_airtable_table_name', 'address', 'categories', 'city', 'country', 'latitude', 'longitude', 'name', 'postalcode', 'province', 'reviews.date', 'reviews.dateadded', 'reviews.dorecommend', 'reviews.id', 'reviews.rating', 'reviews.text', 'reviews.title', 'reviews.usercity', 'reviews.username', 'reviews.userprovince']


In [6]:
agg_df = df.groupby('name').agg({
    'city': 'first',  # Assuming city and country are the same for all entries of the same name
    'country': 'first',
    'reviews.rating': lambda values: list(values),
    'reviews.text': lambda texts: list(texts),
    'reviews.title': lambda titles: list(titles),
}).reset_index()

agg_df.head(3)

,name,city,country,reviews.rating,reviews.text,reviews.title
0,1785 Inn,North Conway,US,"[0.0, 3.0, 5.0, 2.0, 5.0, 4.0, 0.0, 1.0, 3.0, ...",[to share your opinion of this businesswith YP...,"[fantastic, The dinners did not live up to the..."
1,1900 House,Narragansett,US,"[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, ...",[Sandy and Bill are wonderful hosts. Sandy ser...,"[Lovely Bed Breakfast, Our Best BB Experience,..."
2,40 Berkeley Hostel,Boston,US,"[2.0, 4.0, 2.0, 5.0, 3.0, 4.0, 4.0, 4.0, 4.0, ...","[Room was cold, pillow sucked. Jack hammers an...","[MEH, I would definitely stay here again, Grea..."


In [7]:
docs = [{
	'documentId': record['name'],
	'title': f"Reviews for the hotel {record['name']} in {record['city']}.",
	'metadataJson': json.dumps({
        'city': record['city'],
        'country': record['country'],
        'rating': np.mean(record['reviews.rating']),
        'name': record['name']
    }),
	'section': [
        {
            'title': f'Review for {record["name"]} with rating {rating} titled {title}: ', 
            'text': text
        }
        for title,text,rating in zip(record['reviews.title'],record['reviews.text'],record['reviews.rating'])
    ],
} for record in agg_df.to_dict(orient='records')]

In [8]:
customer_id = "<VECTARA_CUSTOMER_ID>"
corpus_id = "<VECTARA_CORPUS_ID>"
api_key = "<VECTARA_API_KEY>"

In [9]:
for doc in docs:
    index_document(customer_id, corpus_id, api_key, doc)

In [10]:
docs[14]

{'documentId': 'Ambassador Inn Albuquerque',
 'title': 'Reviews for the hotel Ambassador Inn Albuquerque in Albuquerque.',
 'metadataJson': '{"city": "Albuquerque", "country": "US", "rating": 3.2, "name": "Ambassador Inn Albuquerque"}',
 'section': [{'title': 'Review for Ambassador Inn Albuquerque with rating 1.0 titled Disappointing: ',
   'text': 'Cheap-quality room in industrial area. No restaurants or other services nearby. Hard bed, sloppy cleaning, few electrical outlets, lots of road noise. Breakfast is stale donuts and usually empty orange-juice dispenser. Bathroom has no towel bars or even hooks, and little space for toiletries. Window is cracked and hard to open or close. TV is old, with pocked glass. On... More'},
  {'title': 'Review for Ambassador Inn Albuquerque with rating 4.0 titled Small but clean Every thing we needed: ',
   'text': 'We enjoyed our stay. We were looking for a cheaper room during the Fiestea. We had a frige and microWave-you need to bring your own coffe